In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Importing Libraries

In [ ]:
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, BatchNormalization, LeakyReLU, Activation
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.utils import np_utils

# Checking CPU and GPU status

In [ ]:
print('CPU: {}'.format(len(tf.config.list_physical_devices('CPU'))))
print('GPU: {}'.format(len(tf.config.list_physical_devices('GPU'))))

# Loading and Analysing the DataSet

In [ ]:
# loading the dataset

data = pd.read_csv('/kaggle/input/facial-expression-recognitionferchallenge/fer2013/fer2013/fer2013.csv')
data.head()

In [ ]:
# shape of the dataframe loaded

data.shape

In [ ]:
# basic analysis

data.info()

In [ ]:
# checking the values for understanding of usage

for i in ['emotion', 'Usage']:
    print(data[i].unique())

In [ ]:
# calculate the image size

math.sqrt(len(data.pixels[0].split(' ')))

In [ ]:
# chaning the pixels to numpy array

X = data['pixels'].apply(lambda x: np.array(x.split()).reshape(48, 48, 1).astype('float32'))
X = np.stack(X, axis=0)
X.shape

In [ ]:
# extracting y label and declaring the emotions

lookup = ('anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise', 'neutral')
y = data['emotion']
y.shape

# Data Visualization

In [ ]:
# Count of photo with respect to different emotions

sns.countplot(x=y)
plt.xticks(range(7), lookup);

In [ ]:
# examine the images

def plot_index(index):
    plt.imshow(X[index].reshape((48,48)), cmap='gray')
    plt.title(lookup[y[index]])

index = int(input('Enter Value: '))
plot_index(index)

# Feature Engineering

In [ ]:
# Label encoding

le = LabelEncoder()
img_labels = le.fit_transform(y)
img_labels = np_utils.to_categorical(img_labels)
img_labels.shape

In [ ]:
# dividing the image value by 255.0 so that all values can be filled between 0.0 and 1.0

print('Before Transformation', np.max(X))
X = X / 255.0
print('After Transformation', np.max(X))

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(img_array, img_labels,
                                                    shuffle=True, stratify=img_labels,
                                                    test_size=0.1, random_state=42)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

In [ ]:
# splitting the dataset into training and testing sets

X_train, X_valid, y_train, y_valid = train_test_split(X, img_labels, test_size=0.1, stratify=img_labels)
print('Shape of training set', y_train.shape[0])
print('Shape of validation set', y_valid.shape[0])

# Creating and Training the Model

In [ ]:
# creating model of convolutional neural network

model = Sequential()

model.add(
    Conv2D(
        filters = 64,
        kernel_size = (5, 5),
        input_shape = (48, 48, 1),
        activation = 'elu',
        padding = 'same',
        kernel_initializer = 'he_normal'
    )
)
model.add(BatchNormalization())

model.add(
    Conv2D(
        filters = 64,
        kernel_size = (5, 5),
        activation = 'elu',
        padding = 'same',
        kernel_initializer = 'he_normal'
    )
)
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.4))

model.add(
    Conv2D(
        filters = 128,
        kernel_size = (3, 3),
        activation = 'elu',
        padding = 'same',
        kernel_initializer = 'he_normal'
    )
)
model.add(BatchNormalization())

model.add(
    Conv2D(
        filters = 128,
        kernel_size = (3, 3),
        activation = 'elu',
        padding = 'same',
        kernel_initializer = 'he_normal'
    )
)
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.4))

model.add(
    Conv2D(
        filters = 256,
        kernel_size = (3, 3),
        activation = 'elu',
        padding = 'same',
        kernel_initializer = 'he_normal'
    )
)
model.add(BatchNormalization())

model.add(
    Conv2D(
        filters = 256,
        kernel_size = (3, 3),
        activation = 'elu',
        padding = 'same',
        kernel_initializer = 'he_normal'
    )
)
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.4))

model.add(Flatten(name='flatten'))
        
model.add(
    Dense(
        128,
        activation='elu',
        kernel_initializer='he_normal',
    )
)
model.add(BatchNormalization())
    
model.add(Dropout(0.6))
    
model.add(
    Dense(
        7,
        activation='softmax'
    )
)

In [ ]:
# checking the summary of the model

model.summary()

In [ ]:
# compile the model

model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [ ]:
# function for early stopping in when change in accuracy is low and ReduceLROnPlateau for learning rate

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.00005,
    patience=11,
    verbose=1,
    restore_best_weights=True,
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=7,
    min_lr=1e-7,
    verbose=1,
)

callbacks = [
    early_stopping,
    lr_scheduler,
]

In [ ]:
# creating images for increasing data

gen = ImageDataGenerator(
    rotation_range = 15,
    width_shift_range = 0.15,
    height_shift_range = 0.15,
    shear_range = 0.15,
    zoom_range = 0.15,
    horizontal_flip = True
)
gen.fit(X_train)

In [ ]:
# training the model

model.fit(
    gen.flow(X_train, y_train, batch_size=32),
    validation_data = (X_valid, y_valid),
    steps_per_epoch = len(X_train) / 32,
    epochs = 100,
    callbacks = callbacks,
    use_multiprocessing = True
)

In [ ]:
#Saving the  model to  use it later on  

fer_json = model.to_json()  
with open("fer.json", "w") as json_file:  
    json_file.write(fer_json)  
model.save_weights("fer.h5")  

# Model Evaluation

In [ ]:
predictions = model.predict(X_valid)
print(classification_report(np.argmax(y_valid, axis=1), np.argmax(predictions, axis=1)))

In [ ]:
plt.figure(figsize = (10,10))
sns.heatmap(confusion_matrix(np.argmax(y_valid, axis=1), np.argmax(predictions, axis=1)),
            xticklabels = lookup,
            yticklabels = lookup,
            annot = True
)